In [24]:
# imports
import os
import cv2
from sklearn.ensemble import BaggingClassifier
#import matplotlib.pyplot as plt
#from sklearn.model_selection import train_test_split
#import sklearn as skl
import sklearn.preprocessing as preproc
#import numpy as np
import keras as k
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D, Flatten, Dense
#import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
#from time import time
import pandas as pd
from imblearn.over_sampling import SMOTENC
import tensorflow as tf

In [7]:
#### variabili globali
image_path = '..\..\skin-cancer-mnist-ham10000\HAM10000_images_part_1'
csv_path = 'HAM10000_metadata.csv'
dataset = pd.read_csv(csv_path, encoding = "ISO-8859-1")

In [44]:
######## API varie

# - carica in memoria immagini e label
# - 1-hot encoding
# - implementazione modello tramite ensamble (nel classificatore va incluso il numero di labels)
# - train e test usando il bagging, ma prima bisogna fare SMOTE su vasc e bkl?



# carica le immagini e le label
def loadPictures(path, dataset):
    X=[];
    y=[];
    
    for _,_,files in os.walk(path+"/"):
        for file in files: 
            #print(file) 
            #if file.find('.jpg')<0:
            #    continue     
            img = cv2.imread(path+"/"+file)
            X.append(img)
            file = file.replace(".jpg", "") #tagliare il .jpg che non compare nel dataset
            y.append(dataset[dataset['image_id'] == file]['dx'])
            
    #conteggio label (serve per l'ultimo livello delle CNN)    
    labels = set(dataset['dx'])
    return X,y,labels


# 1hot encoding delle labels
def encoding(y):
    #Encoding
    #l_enc = preproc.LabelEncoder()
    h_enc = preproc.OneHotEncoder(sparse=False)
    #
    #l_enc.fit(labels)
    #lab1 = l_enc.transform(labels)
    #y1 = l_enc.transform(y)
    #
    #h_enc.fit(lab1.reshape(-1, 1))
    #
    #labs_hot = h_enc.transform(lab1.reshape(-1, 1))
    #y_Hot = h_enc.transform(y1.reshape(-1, 1))
    print(y[0])
    h_enc.fit_transform(y)
    print(y[0])

    #print(labels)
    #print('\n\n')
    #print(lab1)
    #print('\n\n')
    #print(labs_hot)


# conteggio statistiche dataset
def stats(dataset):
    labels = set(dataset['dx'])
    for label in labels:
        print(label, ': ', dataset[dataset['dx']==label]['dx'].count())
        #print (label)

        
        
def new_classifier():
    input_img = Input(shape=(450, 600, 3))  # 3x600x450 image RGB 
    #print (tf.shape(input_img))
    x = Conv2D(20, (5, 5), activation='relu', padding='same')(input_img) # 20x450x600
    #print (tf.shape(x))
    x = MaxPooling2D((2, 2), padding='same')(x) # 20x225x300
    #print (tf.shape(x))
    x = Conv2D(140, (3, 3), activation='relu', padding='same')(x) # 140x75x100
    #print (tf.shape(x))
    x = MaxPooling2D((2, 2), padding='same')(x) # 40x38x50
    #print (tf.shape(x))
    x = Conv2D(50, (3, 3), activation='relu', padding='same')(x) # 50x38x50
    #print (tf.shape(x))
    x = MaxPooling2D((2, 2), padding='same')(x) # 50x19x25
    #print (tf.shape(x))
    x = Flatten()(x) # qui la x diventa monodimensionale
    #print (tf.shape(x))
    # da qualche parte ci va il numero delle label?
    encoded = Dense(7, activation='softmax')(x)
    classifier = k.models.Model(input_img, encoded)   #questo è il nostro base_estimator, compile configura per il training
    classifier.compile(optimizer='adadelta', loss='binary_crossentropy')
    return classifier



def ensemble():
    #da settare 
    #    base_estimator il classificatore come CNN,
    #    n_estimators,
    #    max_samples è il numero di elementi nei training set,
    #    bootstrap se lo vogliamo o no (booleano),
    #    n_jobs per il parallelismo,
    classifier = new_classifier()
    bagging = BaggingClassifier(base_estimator=classifier, n_estimators=15, 
                                bootstrap=True, max_samples=0.5, n_jobs=7)
    return bagging

In [9]:
X,y,labels = loadPictures(image_path, dataset)

In [21]:
b_class = ensemble()

In [45]:
#creazione split
#feat_train, feat_test, target_train, target_test = train_test_split(X, y, test_size=0.20, random_state=28) 
#smote = SMOTENC(sampling_strategy='dict', categorical_features=['bkl', 'vasc'])
#X_sm, y_sm = smote.fit_sample(X,y)
#len(X)
#len(X_sm)

encoding(y)
#print(type(X))

4349    nv
Name: dx, dtype: object


KeyError: 0